In [1]:
from pathlib import Path
import json

ROOT_PATH = Path("")
XAI_PATH = ROOT_PATH / ""

In [2]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)
    
def find_best_and_worst_results(base_path, dict_name='xgboost'):
    best_results = {}
    worst_results = {}
    
    for directory in base_path.iterdir():
        if directory.is_dir() and dict_name in directory.name:
            aggregate_results_path = directory / 'aggregate_results.json'
            
            if aggregate_results_path.exists():
                with open(aggregate_results_path, 'r') as f:
                    data = json.load(f)
                    
                    for result in data:
                        metrics = result['metrics']['mean']
                        iteration = result['iteration']
                        params = result['params']
                        std = result['metrics']['std']
                        
                        for metric in metrics:
                            if metric not in best_results or metrics[metric] > best_results[metric]['score']:
                                best_results[metric] = {
                                    'directory': directory.name,
                                    'iteration': iteration,
                                    'score': metrics[metric],
                                    
                                    'all scores':metrics,
                                    'std': std,
                                    'params': params
                                }
                                
                            if metric not in worst_results or metrics[metric] < worst_results[metric]['score']:
                                worst_results[metric] = {
                                    'directory': directory.name,
                                    'iteration': iteration,
                                    'score': metrics[metric],
                                    'all scores':metrics
                                }
                                
    return best_results, worst_results

In [ ]:
best_results, worst_results = find_best_and_worst_results(XAI_PATH, dict_name='std_6h')

print("Best Results:")
for metric, result in best_results.items():
    if metric == 'roc_auc':
        print(f"Best {metric}:")
        print(json.dumps(result, indent=4))

print("\nWorst Results:")
for metric, result in worst_results.items():
    print(f"Worst {metric}:")
    print(json.dumps(result, indent=4))